In [1]:
%%capture
from numpy import int16
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from components.ovc_actif import actif_in_Q3Q4, actif_in_Q3strict, actif_in_Q4strict, actif_in_Q3, actif_in_Q4
from pandas import concat

from datetime import datetime
date_du_jour = datetime.today().strftime("%d_%m_%Y")

In [2]:
actif_in_Q3.id_patient.count()
actif_in_Q4.id_patient.count()

if not ((actif_in_Q3.id_patient.count() == actif_in_Q3Q4.id_patient.count() + actif_in_Q3strict.id_patient.count()) and (actif_in_Q4.id_patient.count() == actif_in_Q3Q4.id_patient.count() + actif_in_Q4strict.id_patient.count())):
    raise ValueError("Qi <> Qistrict + QiQj, bad logic")
else:
    print("the logic  Qi = Qistrict + QiQj is valid")

12859

16555

the logic  Qi = Qistrict + QiQj is valid


In [3]:
# actif_in_Q3Q4 = actif_in_Q3[actif_in_Q3.id_patient.isin(actif_in_Q4.id_patient)]

#### check the symetrie

actif_in_Q4Q3 = actif_in_Q4[actif_in_Q4.id_patient.isin(actif_in_Q3.id_patient)]
actif_in_Q4Q3.id_patient.count()

9168

In [4]:
actif_in_Q4strict.id_patient.count()
actif_in_Q3strict.id_patient.count()
actif_in_Q3Q4.id_patient.count()

print("Find Parenting:")
actif_in_Q4strict.parenting_detailed.unique()
actif_in_Q3strict.parenting_detailed.unique()
actif_in_Q3Q4.parenting_detailed.unique()

7387

3691

9168

Find Parenting:


array(['no', 'yes', 'has_started'], dtype=object)

array(['no'], dtype=object)

array(['no'], dtype=object)

In [5]:
completed_Q4strict =  actif_in_Q4strict[actif_in_Q4strict.complete_at_least=="yes"]
completed_Q3strict =  actif_in_Q3strict[actif_in_Q3strict.complete_at_least=="yes"]
completed_Q3Q4 =  actif_in_Q3Q4[actif_in_Q3Q4.complete_at_least=="yes"]

completed_Q4strict.id_patient.count()
completed_Q3strict.id_patient.count()
completed_Q3Q4.id_patient.count()

print('Find Parenting:')
completed_Q4strict.parenting_detailed.unique()
completed_Q3strict.parenting_detailed.unique()
completed_Q3Q4.parenting_detailed.unique()

7177

3555

9020

Find Parenting:


array(['no', 'yes', 'has_started'], dtype=object)

array(['no'], dtype=object)

array(['no'], dtype=object)

# Enrolled Q4

In [6]:
enrolledQ4_CQ4strict = completed_Q4strict[completed_Q4strict.isEnrolledQ4=="yes"]
#enrolledQ4_CQ3strict = completed_Q3strict[completed_Q3strict.isEnrolledQ4=="yes"]
enrolledQ4_CQ3Q4 = completed_Q3Q4[completed_Q3Q4.isEnrolledQ4=="yes"]


enrolledQ4_CQ4strict.id_patient.count()
#enrolledQ4_CQ3strict.id_patient.count()
#enrolledQ4_CQ3Q4.id_patient.count()




#enrolledQ4_CQ3Q4[enrolledQ4_CQ3Q4.id_patient.isin(enrolledQ4_CQ4strict.id_patient)].id_patient.count()
#enrolledQ4_CQ3Q4[enrolledQ4_CQ3Q4.id_patient.isin(enrolledQ4_CQ3strict.id_patient)].id_patient.count()
#enrolledQ4_CQ4strict[enrolledQ4_CQ4strict.id_patient.isin(enrolledQ4_CQ3strict.id_patient)].id_patient.count()


#enrolledQ4_CQ4strict[~enrolledQ4_CQ4strict.id_patient.isin(enrolledQ4_CQ3strict.id_patient)].id_patient.count()
#enrolledQ4_CQ3Q4[~enrolledQ4_CQ3Q4.id_patient.isin(enrolledQ4_CQ3strict.id_patient)].id_patient.count()
#enrolledQ4_CQ3Q4[~enrolledQ4_CQ3Q4.id_patient.isin(enrolledQ4_CQ4strict.id_patient)].id_patient.count()


print('find parenting')
enrolledQ4_CQ4strict.parenting_detailed.unique()

1362

find parenting


array(['no'], dtype=object)

# Concatenation

In [7]:
all_ovc = concat([completed_Q3Q4,enrolledQ4_CQ4strict])

In [8]:
all_ovc.drop_duplicates("id_patient",inplace=True)
all_ovc.id_patient.count()
all_ovc.test_results.fillna("0,",inplace=True)
all_ovc.test_results.value_counts()

10382

0,          7393
0,,2,       1847
2,          1115
1,             6
0,,3,          6
0,,1,          5
3,             4
0,,1,,2,       2
0,,2,,3,       2
2,,3,          2
Name: test_results, dtype: int64

In [9]:
ovc_data = all_ovc[
    (all_ovc.test_results!="0,,3,")&
    (all_ovc.test_results!="0,,1,,2,")&
    (all_ovc.test_results!="0,,2,,3,")&
    (all_ovc.test_results!="3,")&
    (all_ovc.test_results!="2,,3,")
]
ovc_data.id_patient.count()
ovc_data.test_results.value_counts()

10366

0,       7393
0,,2,    1847
2,       1115
1,          6
0,,1,       5
Name: test_results, dtype: int64

# postive OVC_DREAMS

In [10]:
positive_dreams= ovc_data[
    (ovc_data.test_results=="1,")|
    (ovc_data.test_results=="0,,1,")
]
positive_dreams.id_patient.count()

11

In [11]:
positive_dreams

,id_patient,id_parenting_group,departement,commune,nbre_pres_for_inter,nbre_parenting_coupe_present,has_comdom_topic,number_of_condoms_sensibilize,number_condoms_reception_in_the_interval,number_test_date_in_the_interval,...,ps_1519,ps_2024,secondary_1014,secondary_1519,secondary_2024,complete_1014,complete_1519,complete_2024,complete_at_least,isEnrolledQ4
672,85541,None,Ouest,Delmas,0,0,no,1,0,0,...,no,no,no,no,no,no,no,no,yes,no
893,80897,None,Artibonite,Grande Saline,0,0,no,1,0,0,...,no,no,no,no,secondary,no,no,complete,yes,no
1785,114955,None,Artibonite,Verrettes,8,0,yes,2,1,0,...,no,no,no,no,no,no,no,no,yes,no
2860,119259,None,Artibonite,Grande Saline,0,0,no,2,0,0,...,no,no,no,no,no,no,no,no,yes,no
3922,109625,None,Artibonite,Petite Rivière de l'Artibonite,0,0,no,1,0,0,...,no,no,no,no,no,no,no,no,yes,no
4630,95319,None,Ouest,Delmas,0,0,no,1,0,0,...,no,no,no,no,no,no,no,no,yes,no
7546,131108,None,Artibonite,Grande Saline,0,0,no,1,0,0,...,no,no,no,no,no,no,no,no,yes,no
7553,131192,None,Artibonite,Grande Saline,0,0,no,1,0,0,...,no,no,no,no,no,no,no,no,yes,no
7565,130124,None,Artibonite,Grande Saline,0,0,no,1,0,0,...,no,no,no,no,no,no,no,no,yes,no
7575,130358,None,Artibonite,Grande Saline,0,0,no,1,0,0,...,no,no,no,no,no,no,no,no,yes,no


# OVC_SERV_DREAMS

In [12]:
ovc_data.ovc_age.unique()
ovc_data.parenting.unique()
ovc_data.parenting_detailed.unique()

array(['10-14', '18-24', '25-29', '15-17', 'not_valid_age'], dtype=object)

array(['no'], dtype=object)

array(['no'], dtype=object)

In [13]:
ovcS_dreams = ovc_data[
    (ovc_data.ovc_age=="10-14")|
    (ovc_data.ovc_age=="15-17")
]
ovcS_dreams.id_patient.count()
ovcS_dreams['Gender'] = "Female"

5297

/var/folders/7v/rs0k237d6nj3vdxfc1h7xtcw0000gn/T/ipykernel_2783/4236012469.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ovcS_dreams['Gender'] = "Female"


In [14]:
ovcS_dreams.parenting_detailed.unique()

array(['no'], dtype=object)

# Format Datim

In [15]:
datim_ovc_dreams = ovcS_dreams.pivot_table(index="commune",values="id_patient",columns=["Gender","ovc_age"],aggfunc='count',fill_value=0)

In [16]:
datim_ovc_dreams[('Female','<1')] = 0
datim_ovc_dreams[('Female','1-4')] = 0
datim_ovc_dreams[('Female','5-9')] = 0
datim_ovc_dreams[('Female','18+')] = 0
datim_ovc_dreams[('Male','<1')] = 0
datim_ovc_dreams[('Male','1-4')] = 0
datim_ovc_dreams[('Male','5-9')] = 0
datim_ovc_dreams[('Male','10-14')] = 0
datim_ovc_dreams[('Male','15-17')] = 0
datim_ovc_dreams[('Male','18+')] = 0

columns_orientation =[
    ('Female','<1'),
    ('Female','1-4'),
    ('Female','5-9'),
    ('Female','10-14'),
    ('Female','15-17'),   
    ('Female','18+'),  
    ('Male','<1'),
    ('Male','1-4'),
    ('Male','5-9'),
    ('Male','10-14'),
    ('Male','15-17'),   
    ('Male','18+'),   
]

datim_ovc_dreams = datim_ovc_dreams.reindex(columns= columns_orientation)
datim_ovc_dreams

Gender                         Female                         Male          \
ovc_age                            <1 1-4 5-9 10-14 15-17 18+   <1 1-4 5-9   
commune                                                                      
Cap-Haïtien                         0   0   0   247   264   0    0   0   0   
Carrefour                           0   0   0   124   112   0    0   0   0   
Delmas                              0   0   0    80   101   0    0   0   0   
Desdunes                            0   0   0    88   109   0    0   0   0   
Dessalines                          0   0   0   188   102   0    0   0   0   
Grande Saline                       0   0   0   101   145   0    0   0   0   
Gressier                            0   0   0    73    41   0    0   0   0   
Kenscoff                            0   0   0   517   510   0    0   0   0   
La Chapelle                         0   0   0    31    34   0    0   0   0   
Liancourt                           0   0   0    42    67   0    0   0   0   
Limonade                            0   0   0    76    75   0    0   0   0   
Milot                               0   0   0    60    39   0    0   0   0   
Montrouis                           0   0   0    49    61   0    0   0   0   
Petite Rivière de l'Artibonite      0   0   0    99   149   0    0   0   0   
Plaine-du-Nord                      0   0   0    72    74   0    0   0   0   
Port-au-Prince                      0   0   0   134   219   0    0   0   0   
Pétionville                         0   0   0    72   105   0    0   0   0   
Quartier-Morin                      0   0   0    33    25   0    0   0   0   
Saint-Marc                          0   0   0   275   365   0    0   0   0   
Tabarre                             0   0   0    82    75   0    0   0   0   
Verrettes                           0   0   0    77   105   0    0   0   0   

Gender                                          
ovc_age                        10-14 15-17 18+  
commune                                         
Cap-Haïtien                        0     0   0  
Carrefour                          0     0   0  
Delmas                             0     0   0  
Desdunes                           0     0   0  
Dessalines                         0     0   0  
Grande Saline                      0     0   0  
Gressier                           0     0   0  
Kenscoff                           0     0   0  
La Chapelle                        0     0   0  
Liancourt                          0     0   0  
Limonade                           0     0   0  
Milot                              0     0   0  
Montrouis                          0     0   0  
Petite Rivière de l'Artibonite     0     0   0  
Plaine-du-Nord                     0     0   0  
Port-au-Prince                     0     0   0  
Pétionville                        0     0   0  
Quartier-Morin                     0     0   0  
Saint-Marc                         0     0   0  
Tabarre                            0     0   0  
Verrettes                          0     0   0

In [17]:
# datim_ovc_dreams.reset_index()

# Save the data

In [18]:
#positive_dreams.to_excel('./ovc_results/positive_data/positive_dreams.xlsx',na_rep="",index=False)
#datim_ovc_dreams.to_excel('./ovc_results/datim_ovc_dreams.xlsx',na_rep="",index=True)

ovcS_dreams.to_excel(f"./ovc_results/valid_ovc_data.xlsx",na_rep="",index=False)
positive_dreams.to_excel(f"./ovc_results/positive_data/positive_dreams_{date_du_jour}.xlsx",na_rep="",index=False)
datim_ovc_dreams.to_excel(f"./ovc_results/datim_ovc_dreams_{date_du_jour}.xlsx",na_rep="",index=True)

